# Pandas 操作指南

这份 notebook 是我在学习、使用 pandas 时记下的一些笔记，发布于此方便自己随时查阅。

In [2]:
import pandas as pd
import numpy as np

In [2]:
# 工具函数

class display:
    """Display HTML representation of multiple objects"""
    
    template = """<div style="font-size: 13px;float:left;margin-right:10px;">{0}</div>"""
    
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        html_list = []
        for var_name in self.args:
            var = eval(var_name)
            if hasattr(var, '_repr_html_'):
                html = "<p>{}<p/>{}".format(var_name, var._repr_html_())
            else:
                html = '<pre>{}</pre>'.format(var_name + '\n\n' + repr(var))
            html_list.append(self.template.format(html))
        return '\n'.join(html_list)
        
    def __repr__(self):
        return '\n\n'.join(self.template.format(repr(eval(a)))
                           for a in self.args)

## Series

Series 是 one-dimensional array-like object，它包含一组数据，以及这组数据中每个元素对应的标签。如下例所示，其中第一列为标签(`index`)，第二列为数据(`values`)。

### 创建 Series

=> 使用数组创建 Series

```python
s = pd.Series([4,5,6,7])
```

=> 访问 index 和 value

```python
s.values, s.index
>>> (array([4, 5, 6, 7]), RangeIndex(start=0, stop=4, step=1))
```

=> 指定 index

```python
s = pd.Series([4,5,6,7], index=['a','b','c','d'])
s.values, s.index
>>> (array([4, 5, 6, 7]), Index(['a', 'b', 'c', 'd'], dtype='object'))
```

创建 Series 时，如果只提供数据，没有提供 index，那么 index 就默认从 0 开始递增。也可以在创建 Series 时明确指定 index。

=> 使用字典创建 Series

此时字典的 key, value 分别作为 Series 的 index, value

```python
dct = {'a': 4, 'b': 5, 'c': 6, 'd': 7}
pd.Series(dct)
>>>
a    4
b    5
c    6
d    7
dtype: int64
```

使用字典创建 Series 时，若指定 index 则只会选 dict 中指定的 key, value 对。

```python
pd.Series(dct, index=['b', 'c', 'e'])
>>>
b    5.0
c    6.0
e    NaN
dtype: float64
```

### 访问 Series

=> 用 index 或下标访问 Series

可以使用 index 来访问，也可以使用下标来访问。在访问时，会先尝试使用 index，如果 index 没有匹配的，会尝试使用下标。

```python
s['a'], s[0]
>>> (4, 4)
```

=> 一次访问多个元素

一次访问多个元素时，返回的结果依然是 Series。

```python
s = pd.Series([4,5,6,7], index=['a','b','c','d'])
s[[1, 2]]
>>> 
b    5
c    6
dtype: int64
```


=> 把 Series 作为字典使用

```python
s = pd.Series([4,5,6,7], index=['a','b','c','d'])

'a' in s
>>> True

s.keys()
>>> Index(['a', 'b', 'c', 'd'], dtype='object')

s.values
>>> array([4, 5, 6, 7])

list(s.items())
>>> [('a', 4), ('b', 5), ('c', 6), ('d', 7)]
```

=> 把 Series 作为一维数组使用

Series 也可以使用类似数组中的切片，切片可以在 index 上进行，也可以在下标上进行。

```python
s = pd.Series([4,5,6,7], index=['a','b','c','d'])

s['a': 'c']
>>>
a    4
b    5
c    6
dtype: int64
    
s[0: 2]
>>>
a    4
b    5
dtype: int64
```

### 使用 `loc`, `iloc` 来对 Series 进行访问

可以使用命名的 index 访问 Series，比如 `s['a']`。也可以通过下标来访问，比如 `s[1]`。那么当 Series 用数字作为 index 时，常常会引起混乱。

```python
s = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
s
>>>
1    a
3    b
5    c
dtype: object


s[1]
>>> 'a'

s[0: 2]
>>>
1    a
3    b
dtype: object
```

`s[1]` 中 `1` 被视为 index，而 `s[0: 2]` 中，使用下标。因为存在以上会导致混乱的情况，Pandas 提供了一些行为更为明确的接口。


=> 始终用 index 来访问 Series:

```python
s.loc[1]
>>> 'a'

s.loc[1: 3]
>>>
1    a
3    b
dtype: object
```

=> 明确使用下标来访问:

```python
s.iloc[1]
>>> 'b'

s.iloc[1:3]
>>>
3    b
5    c
dtype: object
```

In [8]:
display('s2>10', 'np.sqrt(s2)')

<div style="font-size: 13px;float:left;margin-right:10px;">a     True
b     True
c    False
d    False
dtype: bool</div>

<div style="font-size: 13px;float:left;margin-right:10px;">a    3.741657
b    3.872983
c    2.449490
d    2.645751
dtype: float64</div>

In [11]:
display('pd.isnull(s4)', 'pd.notnull(s4)', 's4.isnull()')

<div style="font-size: 13px;float:left;margin-right:10px;">California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool</div>

<div style="font-size: 13px;float:left;margin-right:10px;">California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool</div>

<div style="font-size: 13px;float:left;margin-right:10px;">California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool</div>

两个 Series 相加：

In [12]:
display('s3', 's4', 's3+s4')

<div style="font-size: 13px;float:left;margin-right:10px;">Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64</div>

<div style="font-size: 13px;float:left;margin-right:10px;">California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64</div>

<div style="font-size: 13px;float:left;margin-right:10px;">California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64</div>

Series 和 Series.index 都可以用一个 name，这在和 pandas 的某些函数集成的过程中会有用。

In [13]:
s4.name='population'
s4.index.name='state'
s4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

In [21]:
# masking

display('s5>0.3', 's5<0.8', '(s5>0.3)&(s5<0.8)', 's5[(s5>0.3)&(s5<0.8)]')

<div style="font-size: 13px;float:left;margin-right:10px;">a    False
b     True
c     True
d     True
dtype: bool</div>

<div style="font-size: 13px;float:left;margin-right:10px;">a     True
b     True
c     True
d    False
dtype: bool</div>

<div style="font-size: 13px;float:left;margin-right:10px;">a    False
b     True
c     True
d    False
dtype: bool</div>

<div style="font-size: 13px;float:left;margin-right:10px;">b    0.50
c    0.75
dtype: float64</div>

## DataFrame

DataFrame 像是一个表格，每一列和每一行都有索引，通过行列索引可以对应到每一个单元格。

### 创建 DataFrame

DataFrame 的创建方式有很多，下表进行了总结：

![](https://wangyu-name.oss-cn-hangzhou.aliyuncs.com/superbed/2019/05/23/5ce62e18697df1fd0c34e240.jpg)

_来源于 Python for Data Analysis_

In [28]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'year':  [2000, 2001, 2002, 2001, 2002],
    'pop':   [1.5, 1.7, 3.6, 2.4, 2.9]
}

frame = pd.DataFrame(data)
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


In [29]:
frame = pd.DataFrame(data, columns=['year', 'state', 'pop'])
frame

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


In [30]:
frame = pd.DataFrame(data, columns=['year', 'state', 'pop'],
                     index=['a','b','c','d','e'])
frame

,year,state,pop
a,2000,Ohio,1.5
b,2001,Ohio,1.7
c,2002,Ohio,3.6
d,2001,Nevada,2.4
e,2002,Nevada,2.9


In [31]:
frame.columns

Index(['year', 'state', 'pop'], dtype='object')

In [32]:
display("frame['year']", 'frame.year')

<div style="font-size: 13px;float:left;margin-right:10px;">a    2000
b    2001
c    2002
d    2001
e    2002
Name: year, dtype: int64</div>

<div style="font-size: 13px;float:left;margin-right:10px;">a    2000
b    2001
c    2002
d    2001
e    2002
Name: year, dtype: int64</div>

需要注意的是，如 `frame.pop`，因为 `pop` 是 DataFrame 的一个方法，要想访问 `pop` 这一列，就必须得 `frame['pop']` 这样写。

可以很轻易地给 DataFrame 增加一列并赋值。赋值遵循广播机制。

In [33]:
frame['debt'] = 16
frame

,year,state,pop,debt
a,2000,Ohio,1.5,16
b,2001,Ohio,1.7,16
c,2002,Ohio,3.6,16
d,2001,Nevada,2.4,16
e,2002,Nevada,2.9,16


In [34]:
# 如果使用列表进行赋值，长度要匹配
frame['debt'] = np.arange(5)
frame

,year,state,pop,debt
a,2000,Ohio,1.5,0
b,2001,Ohio,1.7,1
c,2002,Ohio,3.6,2
d,2001,Nevada,2.4,3
e,2002,Nevada,2.9,4


In [35]:
# 也可以使用 Series 进行赋值，此时会根据 index 来匹配
debt = pd.Series([1.2, 3.4, 9.0], index=['c', 'a', 'f'])
frame['debt'] = debt
frame

,year,state,pop,debt
a,2000,Ohio,1.5,3.4
b,2001,Ohio,1.7,NaN
c,2002,Ohio,3.6,1.2
d,2001,Nevada,2.4,NaN
e,2002,Nevada,2.9,NaN


In [36]:
# 可以使用 del 删除某一列

del frame['debt']
frame.columns

Index(['year', 'state', 'pop'], dtype='object')

In [37]:
frame.keys()

Index(['year', 'state', 'pop'], dtype='object')

In [38]:
frame.values

array([[2000, 'Ohio', 1.5],
       [2001, 'Ohio', 1.7],
       [2002, 'Ohio', 3.6],
       [2001, 'Nevada', 2.4],
       [2002, 'Nevada', 2.9]], dtype=object)

DataFrame 底层是 numpy 的多维数组，很多针对数组的方法也都可以运用在 DataFrame 上。

In [39]:
# 进行转置
frame.T

,a,b,c,d,e
year,2000,2001,2002,2001,2002
state,Ohio,Ohio,Ohio,Nevada,Nevada
pop,1.5,1.7,3.6,2.4,2.9


### 对 DataFrame 的访问

- `.loc`  基于标签进行索引
- `.iloc` 基于位置进行索引

=> 访问前 3 行

```python
df.iloc[:3, :]
```

=> 访问前 3 列

```python
df.iloc[:, :3]
```

=> 访问 `year` 列的前 2 行

```python
df.loc[:2, 'year']
```

=> 访问多列

```python
df.loc[:'c', ['state', 'pop']]
```

=> 使用 masked select


```python
# 选择满足 `df['pop'] > 2` 的行
df.loc[df['pop'] > 2, ['pop', 'state']]
```

=> 注意事项

直接在 DataFrame 上使用切片时，是对行进行切片

```python
df['a':'c']
```

不使用切片时，是对列进行访问

```python
df[['pop', 'state']]
```

### 对 DataFrame 的修改

=> 修改单个 cell

```python
name_col_index = list(df.columns).index('name')
 
df.iloc[1, name_col_index] = '老李'
```

### 删除行或列

使用 `drop` 方法可以删除 DataFrame 的行或列，该方法返回一个新的 DataFrame 实例。可以通过 `axis` 参数来说明是删除的维度。

=> 删除行

删除对应 index

```python
df.drop(['a', 'c'])
```

=> 按列名删除列

```python
df.drop(['pop'], axis=1)
```

=> 按索引删除列

```python
df.drop(df.columns[x], axis=1)
```

In [51]:
df = pd.DataFrame([[1,2],[3,4]], columns=['a', 'c'], index=['x', 'y'])
df
>>>
   a  c
x  1  2
y  3  4

   a  c
x  1  2
y  3  4


## Index Objects

Series 的 index 是一个 `pd.Index` 对象，而且 `pd.Index` 是不可变对象。它包含很多方法，详细列表可以参考[这里](https://devdocs.io/pandas~0.23-index/)。



根据这些方法，能够

In [46]:
index = frame.index
new_index = index.append(pd.Index(['f']))
new_index

Index(['a', 'b', 'c', 'd', 'e', 'f'], dtype='object')

In [47]:
new_index.difference(index)

Index(['f'], dtype='object')

In [48]:
frame

,year,state,pop
a,2000,Ohio,1.5
b,2001,Ohio,1.7
c,2002,Ohio,3.6
d,2001,Nevada,2.4
e,2002,Nevada,2.9


### reindex

现在假设要对中各行以 `pop` 递增的顺序排列。这就需要在 DataFrame 上使用 `reindex` 方法。`reindex` 方法接受一个新的数组或者 Index 对象作为参数，DataFrame 中各行会以此重新排列。因此只需要构造出新的 index 即可。

In [49]:
row_index = frame['pop'].argsort().values
new_index = frame.index[row_index]
frame = frame.reindex(new_index)
frame

,year,state,pop
a,2000,Ohio,1.5
b,2001,Ohio,1.7
d,2001,Nevada,2.4
e,2002,Nevada,2.9
c,2002,Ohio,3.6


要想调整列的顺序也可以使用 `reindex`

In [50]:
frame = frame.reindex(columns=['pop', 'state', 'year'])
frame

,pop,state,year
a,1.5,Ohio,2000
b,1.7,Ohio,2001
d,2.4,Nevada,2001
e,2.9,Nevada,2002
c,3.6,Ohio,2002


## 操作 DataFrame

In [51]:
frame_1 = pd.DataFrame(np.random.randn(4, 3), columns=list('abc'), index=list('wxyz'))
frame_2 = pd.DataFrame(np.random.randn(3, 3), columns=list('abd'), index=list('xyz'))
frame_1

,a,b,c
w,0.502710,-0.777302,-1.061957
x,-0.176748,0.615648,-1.786709
y,1.180515,0.253940,0.228130
z,-0.876626,0.408096,1.451826


In [52]:
frame_2

,a,b,d
x,0.065928,-0.402935,-0.074010
y,-1.528736,1.136733,1.349797
z,-0.045845,0.766156,-0.381358


In [53]:
frame_1 + frame_2

,a,b,c,d
w,NaN,NaN,NaN,NaN
x,-0.110820,0.212712,NaN,NaN
y,-0.348221,1.390673,NaN,NaN
z,-0.922471,1.174251,NaN,NaN


两个 DataFrame 直接相加，如果某个单元格并非两个 DataFrame 都有，结果就会是 NaN。为了避免这种局面可以使用 `add` 方法，并提供 `fill_value` 参数。

In [54]:
frame_1.add(frame_2, fill_value=0)

,a,b,c,d
w,0.502710,-0.777302,-1.061957,NaN
x,-0.110820,0.212712,-1.786709,-0.074010
y,-0.348221,1.390673,0.228130,1.349797
z,-0.922471,1.174251,1.451826,-0.381358


这里结果中依然存在一个 NAN，那是因为 fill_value 只在做加和的两者中存在一个缺失时，才会起作用。可以再使用 `fillna` 对缺失值填充。

In [55]:
frame_1.add(frame_2, fill_value=0).fillna(0)

,a,b,c,d
w,0.502710,-0.777302,-1.061957,0.000000
x,-0.110820,0.212712,-1.786709,-0.074010
y,-0.348221,1.390673,0.228130,1.349797
z,-0.922471,1.174251,1.451826,-0.381358


## 关于空值

Python 中使用 None 表示空值，但是 numpy 中 `None` 无法转换为一个数字，因此一个数组中含有 `None` 则会被转换为 object 类型。

对于 object 类型，无法使用 numpy 底层的 C 实现来加速，关于这个数组的所有运算都会在 Python 层面进行，性能会大大受影响。

=> dropna

`df.dropna()` 会将含有空值的所有行都删除掉。`df..dropna(how='all')` 只会删除掉整行都是空值的行。

=> fillna

`df.fillna(0)` 会给将空值填充为 0。但有的时候不同的列，数据类型不同，因此希望填充的值也不同。

传入一个字典，可以对特定的列指定默认填充值。下面的例子中，只会对 age 和 name 两列进行填充。

```python
df.fillna({"age": 10, "name": "unknown"})`
```

`fillna` 默认会返回一个新的 DataFrame 对象，可以使用 `df.fillna(0, inplace=True)` 来在原地进行填充。

还可以通过 `method` 参数指定填充策略。

```python
df.fillna(method='ffill')
```

## 合并 DataFrame

默认 pandas 会在两个 df 中寻找共有的列，然后以此列进行合并。

```python
merged = pd.merge(df1, df2)
```

**how**

默认会采用 `inner join`，可以通过 `how` 来指定 join 的策略，可选项有 `inner`,`outer`,`left`,`right` 。 

```python
merged = pd.merge(df1, df2, how='outer')
```

**on**

如果两个 DataFrame 中存在对个相同的列，可以通过 `on` 来指定使用哪一列来进行合并。

```python
merged = pd.merge(df1, df2, on='name')
```

**left_on / right_on**

有的时候待合并的两个 df 中，对应的列的名字不同，此时就可以通过 `left_on` 和 `right_on` 分别指定作为合并依据的列。

```python
merged = pd.merge(df1, df2, left_on='id', right_on='uid')
```

## Sorting and Ranking

=> sort_index

使用 `sort_index` 按 index 的数字或字母顺序进行排序。

```python
s = pd.Series([0, 1, 2, 3], index=['d', 'a', 'b', 'c'])
s.sort_index()
>>>
a    1
b    2
c    3
d    0
dtype: int64
```

DataFrame 行和列均可以以排序：

```python
# 对 index 进行 sort
df.sort_index()

# 对 columns 进行 sort
df.sort_index(axis=1)
```

可以明确指定使用降序排列：

```python
df.sort_index(ascending=False)
```

=> sort_values

对 DataFrame 常常希望根据某一列的值来对行进行排序，此时可以使用 `by` 指出该列，还可以指定多列：

```python
df.sort_values(by='age')
df.sort_values(by=['age', 'name'])
```

=> rank

`rank` 用来计算一组数在整体中的排名。里面例子中 `-5` 最小，所以排在第一位。对于 7 因为有重复，2 个 7 分别排名为 6,7，则 (6+7)/2=6.5 为最终的排名。

```python
s = pd.Series([7, -5, 7, 1])
s.rank()
>>>
0    3.5
1    1.0
2    3.5
3    2.0
dtype: float64
```

可以通过 `method` 指出排名的取法，默认是平均。

```python
s.rank(method='min')
```

|Method   | Description |
|:--------|:------------|
|'average'| Default: assign the average rank to each entry in the equal group.|
|'min'    | Use the minimum rank for the whole group.|
|'max'    | Use the maximum rank for the whole group.|
|'first'  | Assign ranks in the order the values appear in the data.|

## Unique Values, Value Counts, and Membership

In [59]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
obj.unique()

array(['c', 'a', 'd', 'b'], dtype=object)

In [60]:
obj.value_counts()

a    3
c    3
b    2
d    1
dtype: int64

## 迭代

=> itertuples

这里的 row 是一个 namedtuples。

```python
df = pd.DataFrame({"c1": [0,1,2], "c2": [1,2,3]})

for row in df.itertuples(index=False):
    print(row[0], row[1])
    print(row.c1, row.c2)
```

=> df.apply

`df.apply` 接受一个函数作为参数，对每行或每列运用该函数，这个函数如果返回一个 Series，那么最终结果会是一个 DataFrame，否则结果为 Series。

In [62]:
df.apply(lambda x: np.sum(x), axis=1)

0    1
1    3
2    5
dtype: int64

In [63]:
df.apply(lambda x: np.sum(x), axis=0)

c1    3
c2    6
dtype: int64

## 操作字符串

详情参见文档：[Working with Text Data](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html)

In [64]:
df = pd.DataFrame({'name': ['a','b','c'],
                           'info': ['B|C|D', 'B|D', 'A|C']})

df

,name,info
0,a,B|C|D
1,b,B|D
2,c,A|C


In [65]:
df.columns = df.columns.str.upper()
df

,NAME,INFO
0,a,B|C|D
1,b,B|D
2,c,A|C


In [66]:
df['INFO'] = df['INFO'].str.replace('|', '-')
df

,NAME,INFO
0,a,B-C-D
1,b,B-D
2,c,A-C


In [67]:
df['INFO'].str.find('D')

0    4
1    2
2   -1
Name: INFO, dtype: int64